# Code Translator for FHE
We need to transcribe a lot of code to work with Microsoft's SEAL Library. Therefore, we need a parser that reads in code, parses it lexically, and then outputs it in SEAL's format. Here's an example snippet that we have to parse and convert to SEAL's format.

        tmp0 = data_ptr[0] + data_ptr[7];
        tmp7 = data_ptr[0] - data_ptr[7];
        data_ptr[0] = tmp10 + tmp11;
        data_ptr[4] = tmp10 - tmp11;
        z1 = (tmp12 + tmp13) * 0.541196100;
        data_ptr[2] = z1 + tmp13 * 0.765366865;
        data_ptr[6] = z1 + tmp12 * - 1.847759065;
        z1 = tmp4 + tmp7;
        z5 = (z3 + z4) * 1.175875602;
        tmp4 *= 0.298631336;
        z1 *= -0.899976223;
        z3 += z5;
        data_ptr[7] = tmp4 + z1 + z3;


In [3]:
TEST_LINES="""
tmp0 = data_ptr[0] + data_ptr[7];
tmp7 = data_ptr[0] - data_ptr[7];
data_ptr[0] = tmp10 + tmp11;
data_ptr[4] = tmp10 - tmp11;
z1 = (tmp12 + tmp13) * 0.541196100;
data_ptr[2] = z1 + tmp13 * 0.765366865;
data_ptr[6] = z1 + tmp12 * - 1.847759065;
z1 = tmp4 + tmp7;
z5 = (z3 + z4) * 1.175875602;
tmp4 *= 0.298631336;
z1 *= -0.899976223;
z3 += z5;
data_ptr[7] = tmp4 + z1 + z3;
"""

In [24]:
OPS = ['*', '-', '+', '/']
SPECIAL_OPS = ['*=', '+=', '-=', '/=']




"""
Parses the text line
DEST = ARG1 (OP)  ... (OP) ARGN
"""
def parse_line(line):
    if not '=' in line:
        return
    pline = line
    pline.replace(" ","")
    pline.replace(";","")
    for op in SPECIAL_OPS:
        if op in line:
            arg1 = pline.split(op)[0]
            arg2 = pline.split(op)[1]
            return (op, arg1.strip().replace(";",""), arg2.strip().replace(";",""))
    r = pline.split('=')
    dst = r[0]
    args_op = r[1]
    return dst

SOP_TEXT = {
    '+=': "evaluator.add",
    '-=': "evaluator.sub",
    '*=': "evaluator.multiply",
    '/=': "evaluator.multiply"
}
def convert_expr(expr):
    if not expr:
        return ""
    elif isinstance(expr, tuple):
        if len(expr) == 3:
            a, b, c = expr
            if a in SPECIAL_OPS:
                return "{}({}, {});".format(SOP_TEXT[a], b, c)
            elif a in OPS:
                return "Ciphertext {}{}{}{}"
        if len(expr)


In [26]:
for line in TEST_LINES.splitlines():
    print(parse_line(line))
    print(convert_expr(parse_line(line)))

None

tmp0 

tmp7 

data_ptr[0] 

data_ptr[4] 

z1 

data_ptr[2] 

data_ptr[6] 

z1 

z5 

('*=', 'tmp4', '0.298631336')
evaluator.multiply(tmp4, 0.298631336);
('*=', 'z1', '-0.899976223')
evaluator.multiply(z1, -0.899976223);
('+=', 'z3', 'z5')
evaluator.add(z3, z5);
data_ptr[7] 

